# **Bieżąca Analiza Sentymentu Nagłówków z ~Polskich~ Portali Finansowych**

---

## 1. Prosty model językowy (finBERT) do analizy sentymentów

> Instalacja Torch'a w wersji wykorzystującej CPU

> `pip install --user --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu`

In [1]:
import torch

print(torch.__version__)             # Wersja CPU
print(torch.__file__)                # Ścieżka
print(hasattr(torch, "compiler"))    # Oczekujemy True
print(torch.cuda.is_available())     # Oczekujemy False

2.7.0+cpu
/home/jovyan/.local/lib/python3.11/site-packages/torch/__init__.py
True
False


In [2]:
from transformers import pipeline

test_pipe = pipeline("text-classification", model="ProsusAI/finbert")

#print(pipe("British Pound gained and the stocks rallied."))

sentences = [
    "Stocks surge as market optimism grows amid strong economic data.",
    "Tech stocks plummet following disappointing earnings report.",
    "Oil prices hit a new high, fueled by increased global demand.",
    "The Federal Reserve raises interest rates to combat rising inflation.",
    "Investors react nervously as recession fears mount in the market.",
    "Gold prices fall after a strong US jobs report boosts confidence in the economy.",
    "Bitcoin continues its bullish rally, reaching a new all-time high.",
    "Wall Street sees a sharp decline due to weaker-than-expected earnings results.",
    "Corporate profits exceed forecasts, boosting investor sentiment.",
    "Global supply chain disruptions continue to affect earnings reports for major retailers.",
    "Analysts remain optimistic despite ongoing trade tensions between the US and China.",
    "Major banks report higher-than-expected profits in Q1, sending stocks higher.",
    "Retail stocks underperform amid lower consumer spending in the latest quarter.",
    "The housing market cools off as mortgage rates climb higher.",
    "Investors flock to bonds as uncertainty over the economy persists.",
    "Corporate layoffs rise as companies brace for a slowdown in consumer demand.",
    "The economy shows signs of recovery with GDP growth exceeding expectations.",
    "Technology stocks rebound after a tough quarter, fueled by strong product launches.",
    "Austerity measures lead to a decline in government spending, raising concerns about economic growth.",
    "The market experiences a sell-off as geopolitical tensions escalate globally."
]

n=1
for sentence in sentences:
    print(f"{n} {test_pipe(sentence)}")
    n = n+1

Device set to use cpu


1 [{'label': 'positive', 'score': 0.6837644577026367}]
2 [{'label': 'negative', 'score': 0.9547190070152283}]
3 [{'label': 'positive', 'score': 0.9177501201629639}]
4 [{'label': 'positive', 'score': 0.698737621307373}]
5 [{'label': 'negative', 'score': 0.9415711760520935}]
6 [{'label': 'negative', 'score': 0.6376953125}]
7 [{'label': 'positive', 'score': 0.9208424687385559}]
8 [{'label': 'negative', 'score': 0.9750882387161255}]
9 [{'label': 'positive', 'score': 0.9441881775856018}]
10 [{'label': 'negative', 'score': 0.9692643880844116}]
11 [{'label': 'positive', 'score': 0.919272780418396}]
12 [{'label': 'positive', 'score': 0.936418354511261}]
13 [{'label': 'negative', 'score': 0.9726269841194153}]
14 [{'label': 'positive', 'score': 0.7251573801040649}]
15 [{'label': 'neutral', 'score': 0.59334397315979}]
16 [{'label': 'negative', 'score': 0.9667238593101501}]
17 [{'label': 'positive', 'score': 0.9531128406524658}]
18 [{'label': 'positive', 'score': 0.9167876839637756}]
19 [{'label':

### Błędna klasyfikacja nagłówków № 4 oraz № 14, co daje nam skuteczność na poziomie ok. 90%.*
*przy bardzo małej próbie (18/20 poprawne)

---

## 2. Czytanie kanałów RSS i obliczanie sentymentu

In [3]:
from transformers import pipeline
import feedparser

pipe = pipeline("text-classification", model="ProsusAI/finbert")

yahoo_rss = 'https://finance.yahoo.com/news/rssindex'
guardian_rss = 'https://www.theguardian.com/uk/business/rss'
google_news_rss = 'https://news.google.com/rss/topics/CAAqIQgKIhtDQkFTRGdvSUwyMHZNREpmTjNRU0FtVnVLQUFQAQ?hl=en-US&gl=US&ceid=US%3Aen'
bloomberg_rss = 'https://feeds.bloomberg.com/markets/news.rss'

feed = feedparser.parse(bloomberg_rss)

total_score = 0
num_articles = 0

for i, entry in enumerate(feed.entries):
    print(f'Title: {entry.title}')
    print(f'Link: {entry.link}')
    print(f'Published: {entry.published}')
    print(f'Summary: {entry.get("summary", "No summary available")}')

    sentiment = pipe(entry.title)[0]
    print(f'Sentiment {sentiment["label"]}, Score: {sentiment["score"]}')
    print('-' * 45)

    if sentiment['label'] == 'positive':
        total_score += sentiment['score']
        num_articles += 1
    elif sentiment['label'] == 'negative':
        total_score -= sentiment['score']
        num_articles += 1

final_score = total_score / num_articles
print(f'Overall sentiment: {"Positive" if final_score >= 0.15 else "Negative" if final_score <= -0.15 else "Neutral"} {final_score}')

Device set to use cpu


Title: Stocks Power Global Risk Rally on US-China Truce: Markets Wrap
Link: https://www.bloomberg.com/news/articles/2025-05-11/us-futures-dollar-climb-on-geneva-trade-talks-markets-wrap
Published: Sun, 11 May 2025 22:10:21 GMT
Summary: A sharp rebound in risk-appetite amid a reprieve in the trade war between the US and China spurred a rally in stocks, with hopes for a broader agreement between the world’s two largest economies sending bonds, gold and haven currencies lower.
Sentiment negative, Score: 0.4976571798324585
---------------------------------------------
Title: Aramco Debt Nears 3-Year High as Oil Plunge Strains Earnings
Link: https://www.bloomberg.com/news/articles/2025-05-12/aramco-debt-rises-to-3-year-high-as-oil-plunge-strains-earnings
Published: Mon, 12 May 2025 06:30:58 GMT
Summary: Saudi Aramco’s net debt rose to the highest in almost three years, piling pressure on the finances of the world’s biggest oil exporter even as it cut its massive dividend.
Sentiment negative